In [17]:
import os
from glob import glob
import os
import torch
from torch.utils import data
from PIL import Image
import numpy as np
import pandas as pd
import cv2
import argparse 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from PIL import Image
import pretrainedmodels

In [18]:
train_csv = pd.read_csv('Train_label.csv')
train_csv.head(5)

,FileName,Code
0,e456f93392814dbdb2f81cf6d41faada.png,8
1,6959fcdf3f274c6bab63add45ae25d9b.jpg,8
2,6b6202ec2a8443ec9d427ed8d6657960.png,8
3,3da60c0e4af84643bdb0b335d677b0e3.jpg,8
4,0817c1defbeb4419b70fe138b7f9b64e.png,8


In [19]:
print(train_csv[train_csv['Code'] == '2;15'])
#有[2;15]这样的数据，考虑multilabel
"""
target = torch.zeros(29)
labels = list(target.split())
for label in labels:
    target[label] = 1
"""

                                  FileName  Code
8328  b7dccb7d7faa4dc99a1c7267ff9ac72c.jpg  2;15
8408  90115121fa584eb2b47199ba43c3868c.jpg  2;15
8776  c0660c064d654d0497c38d17398fb66d.jpg  2;15


'\ntarget = torch.zeros(29)\nlabels = list(target.split())\nfor label in labels:\n    target[label] = 1\n'

In [20]:
class cloud_dataset(Dataset):
    def __init__(self, path, transform=transforms.Compose([transforms.Resize((224,224)), transforms.RandomHorizontalFlip(0.5),  
                                                           transforms.ToTensor()])):
        self.img_paths = []
        self.filenames = train_csv['FileName']
        #print(self.filenames)
        self.labels = train_csv['Code']
        for img_path in self.filenames:
            self.img_paths.append(glob(path  + 'train_data' + '/' + img_path))
        self.transform = transform
        print(self.img_paths[0])
        
    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self, index):
        img_path = self.img_paths[index]
        label = self.labels[index]
        target = torch.zeros(29)
        labels = list(label.split(';'))
        for label in labels:
            target[int(label)-1] = 1
#             print(target)
#         label = float(label)
#         label = torch.Tensor([label-1])
        #print(img_path)
#         img = cv2.imread(str(*img_path))#BGR

#         img = cv2.resize(img, (224, 224), cv2.INTER_CUBIC)
        img = Image.open(str(*img_path)).convert('RGB')
        img = self.transform(img)
#         img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
#         img = np.ascontiguousarray(img, dtype=np.float32)  # uint8 to float32
#         img /= 255.0  # 0 - 255 to 0.0 - 1.0
        
        #print(img, label)
#         return torch.from_numpy(img), target.float()
        return img, target.float()

In [21]:
EPOCHS = 30
DEVICE = torch.device("cuda:3")
# criterion = nn.CrossEntropyLoss()
# criterion = nn.BCELoss().cuda()
criterion = nn.BCEWithLogitsLoss().to(DEVICE)

In [22]:
# model_name = 'senet154' # could be fbresnet152 or inceptionresnetv2
# model_cloud = pretrainedmodels.__dict__[model_name](num_classes=29, pretrained='imagenet')

model_cloud = models.resnet50(pretrained=True)
# is_cuda = False
# if torch.cuda.is_available():
#     is_cuda = True
# if is_cuda:
#     model_cloud.cuda()




num_ftrs = model_cloud.fc.in_features
model_cloud.fc = nn.Linear(num_ftrs, 29)

# weights_ = '/home/lwlw/data/df_cloud/weights/'
# latest = weights_ + 'epoch_39.pth'
# checkpoint = torch.load(latest)
# model_cloud.load_state_dict(checkpoint['model_state_dict'])


model_cloud = model_cloud.to(DEVICE)
optimizer_r = optim.SGD(model_cloud.parameters(), lr=0.005, momentum=0.9)

# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [23]:
DataSet = cloud_dataset('/home/lwlw/data/df_cloud/')
dataloader = DataLoader(DataSet,batch_size=128, shuffle=True) 

def train_model(model, device, train_loader, optimizer, epoch):
    best_loss = 1e3
    weights = './weights_res50/'
    best_epoch = 0
    
    for epoch in range(epoch):
        latest = weights + 'epoch_{}.pth'.format(epoch)
        model.train()
        run_loss = 0.0
        run_correct = 0
        total_size = 0
        for batchidx, (data, target) in enumerate(train_loader):
            data = data.to(device)
            target = target.to(device)
            optimizer.zero_grad()
            output = model(data)
#             _, preds = torch.max(output, 1)
#             loss = criterion(output, target.squeeze())
#             print(output.shape)
#             print(target.shape)
#             output[output < 0.0] = 0.0
#             output[output > 1.0] = 1.0
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
#             run_loss += loss.item() * data.size(0)
            run_loss += loss.item() *data.size(0)
            total_size += data.size(0)
            #print(data.size(0))
#             run_correct += torch.sum(preds == target.data)
#             print("epoch{} batch{} is done".format(epoch, batchidx))
            
            #len(data)返回一个batch的数据集长度
           #len(train_loader)返回一个epoch要运行多少个batch,即iteration
            if batchidx % 10 == 0:
            #len(train_loader.dataset)返回的是整个数据集的长度
#                 print(len(train_loader.dataset))
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tAverage loss: {:.6f}'.format(
                epoch, batchidx * len(data), len(train_loader.dataset),
                100. * batchidx / len(train_loader), run_loss / total_size))
        epoch_loss = run_loss / len(train_loader.dataset)
#         epoch_acc = run_correct.double() / total_size 
        torch.save({
#         'epoch': epoch,
        'model_state_dict': model.state_dict(),
#         'optimizer_state_dict': optimizer.state_dict(),
#         'loss': loss,
         }, latest)
#         if  epoch_acc > best_acc:
#                 best_acc = epoch_acc
#                 best_epoch = epoch
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            best_epoch = epoch
        print('epoch{} Loss: {:.4f} best_epoch:{}'.format(
                epoch, epoch_loss, best_epoch))
        
        

['/home/lwlw/data/df_cloud/train_data/e456f93392814dbdb2f81cf6d41faada.png']


In [24]:
train_model(model_cloud, DEVICE, dataloader, optimizer_r, 
                      epoch=50)

Train Epoch: 0 [0/10665 (0%)]	Average loss: 0.728274
Train Epoch: 0 [1280/10665 (12%)]	Average loss: 0.531505
Train Epoch: 0 [2560/10665 (24%)]	Average loss: 0.374331
Train Epoch: 0 [3840/10665 (36%)]	Average loss: 0.299485
Train Epoch: 0 [5120/10665 (48%)]	Average loss: 0.259034
Train Epoch: 0 [6400/10665 (60%)]	Average loss: 0.233265
Train Epoch: 0 [7680/10665 (71%)]	Average loss: 0.215841
Train Epoch: 0 [8960/10665 (83%)]	Average loss: 0.202620
Train Epoch: 0 [10240/10665 (95%)]	Average loss: 0.192828
epoch0 Loss: 0.1909 best_epoch:0
Train Epoch: 1 [0/10665 (0%)]	Average loss: 0.124904
Train Epoch: 1 [1280/10665 (12%)]	Average loss: 0.122198
Train Epoch: 1 [2560/10665 (24%)]	Average loss: 0.120979
Train Epoch: 1 [3840/10665 (36%)]	Average loss: 0.120169
Train Epoch: 1 [5120/10665 (48%)]	Average loss: 0.119855
Train Epoch: 1 [6400/10665 (60%)]	Average loss: 0.119585
Train Epoch: 1 [7680/10665 (71%)]	Average loss: 0.118943
Train Epoch: 1 [8960/10665 (83%)]	Average loss: 0.118242
Train

Train Epoch: 15 [0/10665 (0%)]	Average loss: 0.069680
Train Epoch: 15 [1280/10665 (12%)]	Average loss: 0.079761
Train Epoch: 15 [2560/10665 (24%)]	Average loss: 0.080140
Train Epoch: 15 [3840/10665 (36%)]	Average loss: 0.079861
Train Epoch: 15 [5120/10665 (48%)]	Average loss: 0.078904
Train Epoch: 15 [6400/10665 (60%)]	Average loss: 0.078594
Train Epoch: 15 [7680/10665 (71%)]	Average loss: 0.078081
Train Epoch: 15 [8960/10665 (83%)]	Average loss: 0.078191
Train Epoch: 15 [10240/10665 (95%)]	Average loss: 0.078240
epoch15 Loss: 0.0782 best_epoch:15
Train Epoch: 16 [0/10665 (0%)]	Average loss: 0.075061
Train Epoch: 16 [1280/10665 (12%)]	Average loss: 0.076032
Train Epoch: 16 [2560/10665 (24%)]	Average loss: 0.075825
Train Epoch: 16 [3840/10665 (36%)]	Average loss: 0.076348
Train Epoch: 16 [5120/10665 (48%)]	Average loss: 0.076701
Train Epoch: 16 [6400/10665 (60%)]	Average loss: 0.076771
Train Epoch: 16 [7680/10665 (71%)]	Average loss: 0.076617
Train Epoch: 16 [8960/10665 (83%)]	Average l

Train Epoch: 29 [10240/10665 (95%)]	Average loss: 0.064029
epoch29 Loss: 0.0641 best_epoch:29
Train Epoch: 30 [0/10665 (0%)]	Average loss: 0.065687
Train Epoch: 30 [1280/10665 (12%)]	Average loss: 0.062677
Train Epoch: 30 [2560/10665 (24%)]	Average loss: 0.064175
Train Epoch: 30 [3840/10665 (36%)]	Average loss: 0.063127
Train Epoch: 30 [5120/10665 (48%)]	Average loss: 0.063744
Train Epoch: 30 [6400/10665 (60%)]	Average loss: 0.063286
Train Epoch: 30 [7680/10665 (71%)]	Average loss: 0.063175
Train Epoch: 30 [8960/10665 (83%)]	Average loss: 0.063171
Train Epoch: 30 [10240/10665 (95%)]	Average loss: 0.063029
epoch30 Loss: 0.0628 best_epoch:30
Train Epoch: 31 [0/10665 (0%)]	Average loss: 0.066873
Train Epoch: 31 [1280/10665 (12%)]	Average loss: 0.063139
Train Epoch: 31 [2560/10665 (24%)]	Average loss: 0.062266
Train Epoch: 31 [3840/10665 (36%)]	Average loss: 0.061569
Train Epoch: 31 [5120/10665 (48%)]	Average loss: 0.062470
Train Epoch: 31 [6400/10665 (60%)]	Average loss: 0.061886
Train Ep

Train Epoch: 44 [7680/10665 (71%)]	Average loss: 0.046620
Train Epoch: 44 [8960/10665 (83%)]	Average loss: 0.046754
Train Epoch: 44 [10240/10665 (95%)]	Average loss: 0.047008
epoch44 Loss: 0.0470 best_epoch:44
Train Epoch: 45 [0/10665 (0%)]	Average loss: 0.052393
Train Epoch: 45 [1280/10665 (12%)]	Average loss: 0.048153
Train Epoch: 45 [2560/10665 (24%)]	Average loss: 0.046920
Train Epoch: 45 [3840/10665 (36%)]	Average loss: 0.045768
Train Epoch: 45 [5120/10665 (48%)]	Average loss: 0.045684
Train Epoch: 45 [6400/10665 (60%)]	Average loss: 0.045601
Train Epoch: 45 [7680/10665 (71%)]	Average loss: 0.045273
Train Epoch: 45 [8960/10665 (83%)]	Average loss: 0.045197
Train Epoch: 45 [10240/10665 (95%)]	Average loss: 0.045601
epoch45 Loss: 0.0457 best_epoch:45
Train Epoch: 46 [0/10665 (0%)]	Average loss: 0.041776
Train Epoch: 46 [1280/10665 (12%)]	Average loss: 0.046195
Train Epoch: 46 [2560/10665 (24%)]	Average loss: 0.044635
Train Epoch: 46 [3840/10665 (36%)]	Average loss: 0.044099
Train Ep

In [10]:
sub = pd.read_csv('submit_example.csv')
sub.head(5)

,FileName,type
0,0007cab6150d49929780dace45e24fe5.jpg,1
1,001e089b5d154bcd8e6fa67e16aa8e1b.jpg,1
2,002d8e9ed8a94143856edec5ece148f8.png,1
3,00449e99a6c949aeaadc35a8e02ef11a.jpg,1
4,00622e856b3041dfba3dcceb1d378cf6.jpg,1


In [16]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
class test_dataset(Dataset):
    def __init__(self, path):
        self.img_paths = []
        self.filenames = sub['FileName']
        #print(self.filenames)
#         self.labels = train_csv['Code']
        for img_path in self.filenames:
            self.img_paths.append(glob(path  + 'test_data' + '/' + img_path))
        print(self.img_paths[0])
    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self, index):
        img_path = self.img_paths[index]
#         label = self.labels[index]
#         target = torch.zeros(29)
#         labels = list(label.split(';'))
#         for label in labels:
#             target[int(label)-1] = 1
#         label = float(label)
#         label = torch.Tensor([label-1])
        #print(img_path)
        img = cv2.imread(str(*img_path))#BGR
        img = cv2.resize(img, (224, 224), cv2.INTER_CUBIC)
        img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
        img = np.ascontiguousarray(img, dtype=np.float32)  # uint8 to float32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        #print(img, label)
        return torch.from_numpy(img)

thresh = 0.4

def get_result(model, submission, test_loader):
    labels = []
    model.eval()
    device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
    test = submission.copy()
    for batch, data in enumerate(test_loader):
        data = data.to(device)
        output = torch.sigmoid(model(data))
        
        for scores in output:
#             print(scores)
            pred = []
#             pred.append(str(scores[torch.gt(scores,thresh)]))
#             print(scores)
            for i, score in enumerate(scores):
                
                if torch.gt(score,thresh):
                    pred.append(str(i+1))
#                     print(score.index)
            if len(pred) == 0:
                pred.append(str((output.argmax()+1).cpu().numpy()))
#             print(pred)
        labels.append(';'.join(pred))
    sub['type'] = labels
    sub.to_csv('submit_res34.csv',index=False)
#         print(labels)
#         print(';'.join(pred))
#         print(labels)
    
    """
    multi-label的预测方法：
    target = []
    for sample_pred in result:
        pred = []
        for i,score in enumerate(sample_pred):
            if score > cfg['thres'][i]:
                pred.append(str(i))
        if len(pred) == 0:
            pred.append(str(sample_pred.argmax()))
        target.append(' '.join(pred))

    dataset['Predicted'] = target
    dataset.to_csv('submit.csv',index=False)
    """
    
test_dataset = test_dataset('/home/lwlw/data/df_cloud/')
test_dataloader = DataLoader(test_dataset, batch_size=1, num_workers=0,shuffle=False)
model = models.resnet34(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 29)
weights = '/home/lwlw/data/df_cloud/weights_res34/'
latest = weights + 'epoch_32.pth'
checkpoint = torch.load(latest)
model.load_state_dict(checkpoint['model_state_dict'])

model = model.to(device)
get_result(model, sub, test_dataloader)



['/home/lwlw/data/df_cloud/test_data/0007cab6150d49929780dace45e24fe5.jpg']


In [ ]:
x = torch.arange(5)
x = torch.tensor([2,3,1,6,7,9])
# y = torch.sigmoid(x)
# print(y)
# print(x)

# mask = torch.gt(x,1)
# for i, s in enumerate(x):
#     if s % 2:
#         print(i)
print(';'.join(x))

In [3]:
import torch
torch.cuda.device_count()

8

In [2]:
print(pretrainedmodels.model_names)

['fbresnet152', 'bninception', 'resnext101_32x4d', 'resnext101_64x4d', 'inceptionv4', 'inceptionresnetv2', 'alexnet', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'inceptionv3', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19_bn', 'vgg19', 'nasnetamobile', 'nasnetalarge', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn131', 'dpn107', 'xception', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'cafferesnet101', 'pnasnet5large', 'polynet']
